# atomman.defect.Stroh Class

- - -

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.

__Chandler A. Becker__, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), _Office of Data and Informatics, NIST_.

__Zachary T. Trautt__, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), _Materials Measurement Science Division, NIST_.

Version: 2017-01-24

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The atomman.defect.Stroh class solves the Eshelby anisotropic elasticity solution for a perfectly straight dislocation or crack using the Stroh method. This allows for the evaluation of a dislocation's elastic energy, as well as the ability to generate a dislocation monopole system. 

More on the underlying theory can be found in [atomman.defect.Stroh.theory.pdf](atomman.defect.Stroh.theory.pdf).

In performing the calculation, the defect line is inserted at the Cartesian xy = (0,0) position. If the supplied Burgers vector is in the xz plane, then the resulting defect is a dislocation. Burgers vectors parallel to the z-axis will generate screw dislocations, and Burgers vectors parallel to the x-axis will generate edge dislocations. Having the Burgers vector outside of the xz plane will create a split in the system (i.e. form a crack).

The underlying code can be found in [atomman/defect/Stroh.py](https://github.com/usnistgov/atomman/blob/master/atomman/defect/Stroh.py).

## Class Methods of atomman.defect.Stroh

- [\__init__(self, C, burgers, axes=None, tol=1e-8)](atomman.defect.Stroh.__init__.ipynb) -- class initializer.

- [solve(self, C, burgers, axes=None, tol=1e-8)](atomman.defect.Stroh.solve.ipynb) -- calculates the Stroh solution for the given elastic constants and Burgers vector.

- [displacement(self, pos)](atomman.defect.Stroh.displacement.ipynb) -- calculates the Stroh displacement at one or more positions.

- [stress(self, pos)](atomman.defect.Stroh.stress.ipynb) -- calculates the Stroh stress state at one or more positions.

- [eta(self, pos)](atomman.defect.Stroh.eta.ipynb) -- computes the position dependent eta factor. This is used by the displacement and stress methods.

## Class Attributes of atomman.defect.Stroh

- [preln](atomman.defect.Stroh.preln.ipynb) -- the pre-ln strain energy factor of the defect.

- [K_coeff](atomman.defect.Stroh.K_coeff.ipynb) -- the energy coefficient for the defect.

- [K_tensor](atomman.defect.Stroh.K_tensor.ipynb) -- the energy coefficient tensor for the defect.

- [p](atomman.defect.Stroh.p.ipynb) -- the six eigenvalues associated with the Stroh solution.

- [A](atomman.defect.Stroh.A.ipynb) -- the A eigenvectors associated with the Stroh solution.

- [L](atomman.defect.Stroh.L.ipynb) -- the L eigenvectors associated with the Stroh solution.

- [k](atomman.defect.Stroh.k.ipynb) -- the k normalization factors associated with the Stroh solution.

__Docs Navigation:__

Tutorial:

1. [Basics](../tutorial/1 Basics.ipynb)

2. [LAMMPS Functionality](../tutorial/2 LAMMPS Functionality.ipynb)

3. [Defect Generation and Evaluation](../tutorial/3 Defect Generation and Evaluation.ipynb)


Reference:

- [atomman](../reference/atomman.ipynb)

- [atomman.convert](../reference/atomman.convert.ipynb)

- [atomman.defect](../reference/atomman.defect.ipynb)

- [atomman.lammps](../reference/atomman.lammps.ipynb)

- [atomman.tools](../reference/atomman.tools.ipynb)

- [atomman.unitconvert](../reference/atomman.unitconvert.ipynb)